In [1]:
%%capture
!pip install contractions
!pip install -U kaleido
!pip install afinn
!pip install vaderSentiment
!pip install pyabsa==1.16.24
!pip install transformers==4.29.0

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/amazon')

Mounted at /content/drive


In [3]:
import os
import re
import ast
import math
import nltk
import spacy
import torch
import string
import random
import contractions
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import dataset_utils
import absa_utils

from collections import Counter
from gensim.models import CoherenceModel
from sklearn.feature_extraction.text import CountVectorizer

from pyabsa import ATEPCCheckpointManager
from pyabsa import available_checkpoints

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen impo

Version 1.16.24 of pyabsa is outdated. Version 2.4.0 was released Wednesday December 27, 2023.
check release notes at https://github.com/yangheng95/PyABSA/blob/release/release-note.json


<frozen importlib._bootstrap>:671: ImportWarning: APICoreClientInfoImportHook.exec_module() not found; falling back to load_module()
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Setting seed

In [4]:
# Set the random seed for reproducibility.
random_seed = 1
torch.manual_seed(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Constants

In [5]:
PREPROCESSED_DATASET_PATH = "/content/drive/MyDrive/amazon/dataset/dataset.csv"
ABSA_REVIEWS_PATH = "/content/drive/MyDrive/amazon/dataset/absa_reviews.csv"
ABSA_DATASET_PATH = "/content/drive/MyDrive/amazon/dataset/absa_dataset.csv"
ABSA_PROCESSED_DATASET_PATH = "/content/drive/MyDrive/amazon/dataset/absa_proc_dataset.csv"
DATASET_WITH_SENTIMENT_PATH = "/content/drive/MyDrive/amazon/dataset/dataset_with_sentiment.csv"

# Import the dataset

In [6]:
dataset = pd.read_csv(DATASET_WITH_SENTIMENT_PATH)
dataset

<ipython-input-6-2a2ec133bdbd>:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(DATASET_WITH_SENTIMENT_PATH)
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.

,rating,opinion,predictedSentiment,numericSentiment,reviewTime,unixReviewTime,reviewText,reviewLength,summary,summaryLength,...,title,description,price,reviewerID,reviewerName,verified,vote,style,image,imageURLHighRes
0,5,positive,positive,1,2010-02-14,1266105600,I usually keep a few of these around for a qui...,78,Preferred Brand,2,...,"Dole Mandarin in Orange Gel, 4.3-Ounce Cups (...",['Mandarin Orange Gel individual serving in pl...,34.08,A3BH49ZKESHDID,Fred Camfield,False,NaN,NaN,NaN,['https://images-na.ssl-images-amazon.com/imag...
1,5,positive,positive,1,2015-12-16,1450224000,"Fits nice in lunch pack, clean water no taste....",10,clean water no taste. H2O,5,...,"Icelandic Glacial Natural Spring Water, 500 Mi...",['Icelandic glacial natural spring water from ...,35.99,A3PJYWWN9DHVIH,John,True,NaN,"{'Size:': ' 330 Milliliter', 'Flavor:': ' 30 C...",NaN,['https://images-na.ssl-images-amazon.com/imag...
2,5,positive,positive,1,2016-03-25,1458864000,We are vegan. We use this to thicken up sauces...,12,Five Stars,2,...,Guar Gum 8 Ounces (Pack of 2),"[""Pure PowderNatural ThickenerA Gluten-Free Fo...",14.97,A1NCD09GCSJGZD,Test Products &amp;amp;amp; Avid Reader,True,NaN,NaN,NaN,[]
3,2,negative,negative,-1,2017-08-08,1502150400,I ordered these mints 2 times before & loved t...,60,"Need to exchange, if possible",5,...,Sugar-Free Mints - Peppermint Flavor (100 tabs...,"['', '<strong>Features</strong>', '', '', '', ...",5.75,A3AZ2PWY6UULE8,Patricia L. Titzer,True,NaN,NaN,NaN,['https://images-na.ssl-images-amazon.com/imag...
4,4,positive,positive,1,2013-06-01,1370044800,"Price is ok, taste is good. Well worth the ti...",44,Got it for long term storage.,6,...,"Keystone Meats All Natural Canned Pork, 28 Ounce",['Keystone Meats is an OHIO PROUD United State...,11.97,A3MDA891A1EPT8,Chris W.,False,5.0,{'Size:': ' 14.5 Ounce'},NaN,['https://images-na.ssl-images-amazon.com/imag...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752149,5,positive,positive,1,2013-08-30,1377820800,We have tried all of the nut mixes. This one i...,31,Probably the best tasting mixes,5,...,"Planters Deluxe Mixed Nuts, Salted, 34 Ounce Jar","[""Nuts are the heroes of wholesome snacking. A...",17.98,A31UVXEANHGDSL,DavidA,True,NaN,NaN,NaN,[]
752150,5,positive,positive,1,2015-07-27,1437955200,mmmm,1,Five Stars,2,...,REESE'S Pieces Peanut Butter Candy (Pack of 18),"[""Make snack time sweet with REESE'S PIECES Ca...",35.09,AI5YDJXUZUTJK,CB63SK,True,NaN,{'Size:': ' 10.5 Ounce (Pack of 6)'},NaN,['https://images-na.ssl-images-amazon.com/imag...
752151,5,positive,positive,1,2016-04-26,1461628800,these candies are amazing! The lemon is delici...,32,delicious candies!!,2,...,Hand Crafted Candy Tin Lemon Ice Chips Candy 1...,"['Ice chips Candy Lemon, 1. 76 oz. Made out of...",5.77,A1WOAM7BQ0YR43,Stephanie,False,NaN,NaN,NaN,['https://images-na.ssl-images-amazon.com/imag...
752152,4,positive,positive,1,2014-11-23,1416700800,Delicious.,1,Four Stars,2,...,"Kettle Brand Potato Chips, Jalapeno, Single-Se...",['Jalapeno Kettle Brand Potato Chips have a ze...,29.98,A2YSFUH31GVNCX,Norman E. Paddock,True,NaN,NaN,NaN,['https://images-na.ssl-images-amazon.com/imag...


In [7]:
new_dataset = dataset[600000:]
new_dataset = absa_utils.preprocess_review_dataset(new_dataset)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bo

In [8]:
#new_dataset.to_csv(ABSA_DATASET_PATH, encoding='utf-8', index=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
#preprocessed_reviews = new_dataset['preprocessedReviewText'].values
#preprocessed_reviews_df = pd.DataFrame(preprocessed_reviews)
#preprocessed_reviews_df.to_csv(ABSA_REVIEWS_PATH, encoding='utf-8', index=False)

# Aspect-based sentiment analysis

In [10]:
#new_dataset = pd.read_csv(ABSA_DATASET_PATH)

In [11]:
new_dataset = absa_utils.get_reviews_aspects(new_dataset)

Output hidden; open in https://colab.research.google.com to view.

In [12]:
new_dataset.to_csv(ABSA_PROCESSED_DATASET_PATH, encoding='utf-8', index=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bo

In [13]:
new_dataset

,rating,opinion,predictedSentiment,numericSentiment,reviewTime,unixReviewTime,reviewText,reviewLength,summary,summaryLength,...,reviewerID,reviewerName,verified,vote,style,image,imageURLHighRes,preprocessedReviewText,aspects,sentiments
600000,5,positive,positive,1,2015-09-12,1442016000,favorite,1,i prefer this one for their regular coffee,8,...,AA70E3KYP87JL,Sparks n Ashes,True,NaN,{'Flavor:': ' Classico'},NaN,['https://images-na.ssl-images-amazon.com/imag...,favorite,[],[]
600001,5,positive,positive,1,2016-06-10,1465516800,Very gentle on the system and very beneficial ...,12,HIGHLY RECOMMEND,2,...,A2RK5N2B0EDUWO,GA_PEACH,True,NaN,NaN,NaN,['https://images-na.ssl-images-amazon.com/imag...,Very gentle on the system and very beneficial ...,[body],[Positive]
600002,5,positive,positive,1,2014-06-10,1402358400,Great fruit spread. Not too sweet. Not sweeten...,20,Great product,2,...,AW9CJ7C3F9NCR,Sharon Shanahan,True,2.0,{'Format:': ' Grocery'},NaN,['https://images-na.ssl-images-amazon.com/imag...,Great fruit spread. Not too sweet. Not sweeten...,"[fruit spread, sugar, spread]","[Positive, Neutral, Positive]"
600003,5,positive,positive,1,2015-05-03,1430611200,Always fantastic. We've ordered multiple time...,23,Can't go wrong. Love it.,5,...,A2L6DZGP00J2JU,R. Ford,True,NaN,NaN,NaN,['https://images-na.ssl-images-amazon.com/imag...,Always fantastic. We have ordered multiple tim...,"[flavor, delivered]","[Positive, Positive]"
600004,5,positive,positive,1,2017-04-19,1492560000,I adore this honey,4,I'm in love,3,...,AC88UY0E4URNR,Ligiaka,True,NaN,NaN,NaN,['https://images-na.ssl-images-amazon.com/imag...,I adore this honey,[honey],[Positive]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752149,5,positive,positive,1,2013-08-30,1377820800,We have tried all of the nut mixes. This one i...,31,Probably the best tasting mixes,5,...,A31UVXEANHGDSL,DavidA,True,NaN,NaN,NaN,[],We have tried all of the nut mixes. This one i...,"[nut mixes, fat, salt]","[Positive, Negative, Negative]"
752150,5,positive,positive,1,2015-07-27,1437955200,mmmm,1,Five Stars,2,...,AI5YDJXUZUTJK,CB63SK,True,NaN,{'Size:': ' 10.5 Ounce (Pack of 6)'},NaN,['https://images-na.ssl-images-amazon.com/imag...,mm,[],[]
752151,5,positive,positive,1,2016-04-26,1461628800,these candies are amazing! The lemon is delici...,32,delicious candies!!,2,...,A1WOAM7BQ0YR43,Stephanie,False,NaN,NaN,NaN,['https://images-na.ssl-images-amazon.com/imag...,these candies are amazing! The lemon is delici...,"[candies, lemon, lemonade]","[Positive, Positive, Positive]"
752152,4,positive,positive,1,2014-11-23,1416700800,Delicious.,1,Four Stars,2,...,A2YSFUH31GVNCX,Norman E. Paddock,True,NaN,NaN,NaN,['https://images-na.ssl-images-amazon.com/imag...,Delicious.,[],[]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
